In [142]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockaing data:')
# print(df)
global_df

Global blockaing data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,573.0000,3.125000,3.478019e-03,SHA-256,2.979480,150.785340,0.010869,1.638857,0.550048
BTC,666.0000,3.163110,1.000000e+00,SHA-256,736.253294,129.729730,3.163110,410.349445,0.557348
DOGE,63.0000,10000.000000,2.719336e-06,Scrypt,0.002243,1371.428571,0.027193,37.293748,16624.132574
KAS,0.9996,65.461151,9.272818e-07,kHeavyHash,1.359523,86434.573830,0.000061,5.246659,3.859192
LTC,144.0000,6.250000,1.305660e-03,Scrypt,0.002275,600.000000,0.008160,4.896225,2152.191802
RVN,59.0000,2500.000000,1.500000e-07,KawPow,0.000008,1464.406780,0.000375,0.549153,67317.101767


In [152]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,paying_btc_eh_day_$,daily_consumption_btc
SCRYPT,1.081482e-04,16758.651369,"$1,650,357,463.98",1.812418
SHA256ASICBOOST,7.118692e+00,0.539200,"$53,099.31",3.838399
KAWPOW,8.029249e-07,78688.123002,"$7,749,044,255.72",0.063181
KHEAVYHASH,5.528257e-02,4.450899,"$438,315.32",0.246057


In [154]:
max_market_utilisation = 0.76

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,8.219262e-05,1.377437,0.057393,"$5,651.97"
SHA256ASICBOOST,5.410206e+00,2.917184,0.121549,"$11,969.93"
KAWPOW,6.102229e-07,0.048017,0.002001,$197.03
KHEAVYHASH,4.201475e-02,0.187003,0.007792,$767.32


In [145]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_hashrate_eh_s_capped
BCH,SHA-256,2.979480,5.372492e+00,1.803164,4.469220e-01
BTC,SHA-256,736.253294,5.372492e+00,0.007297,5.372492e+00
DOGE,Scrypt,0.002243,8.388177e-05,0.037391,8.388177e-05
KAS,kHeavyHash,1.359523,4.144764e-02,0.030487,4.144764e-02
LTC,Scrypt,0.002275,8.388177e-05,0.036871,8.388177e-05
RVN,KawPow,0.000008,6.036257e-07,0.073995,6.036257e-07


In [146]:
min_order_amount_btc = 0.0001 # BTC
order_duration = 3600  # 1 hour
paying_btc_eh_day = 0.540240 # BTC / EH / day

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['speed_eh_s'] = min_order_amount_btc / (nh_market_df['paying_btc_eh_day'] * order_duration)
speed_df

,speed_eh_s
SCRYPT,1.731065e-12
SHA256ASICBOOST,5.147364e-08
KAWPOW,3.557021e-13
KHEAVYHASH,6.242924e-09
